In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !%cd /content/drive/MyDrive/zeroshot_new_paper

In [36]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/zeroshot_new_paper')

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# === KeyZeroBERTo ===

# Dependencies

In [37]:
# !pip install transformers
# !pip install keybert==0.5.0
# !pip install datasets
# !pip install flair|
# %env PYTORCH_ENABLE_MPS_FALLBACK=1

In [38]:
# !pip install pandarallel

In [39]:
# !pip install evaluate

In [179]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import os

import datasets_handler
import evaluation_metrics
import zeroberto
# def getAgora():
#     return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Data Prep

## Source

In [180]:
# which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
which_dataset = 'ag_news'

if which_dataset=='folhauol':
  arq = '/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_clean_df_articles.csv'
  # arq = '/content/drive/MyDrive/folhauol_clean_df_articles.csv'
  raw_data = pd.read_csv(arq)
  raw_data['full_text'] = raw_data['title'].astype(str)+raw_data['text'].astype(str)
  data_col = 'full_text'
  class_col = 'category'
  hyp_template = "O tema principal deste texto é {}."
  raw_data[class_col] = raw_data[class_col].map(datasets_handler.dict_classes_folha)
#raw_data.head(2)
if which_dataset == 'bbc-news':
  #o dataset bbc-news não tem coluna de título 
  # arq = '/content/drive/MyDrive/zeroshot_new_paper/bbc_news_train_plus_test.csv'
  arq = '/Users/alealcoforado/Documents/Projetos/Datasets/bbc-news/BBC News Train.csv'
  raw_data = pd.read_csv(arq, sep = ',')
  data_col = 'Text'
  class_col = 'Category'
  hyp_template = "this article is about {}."
# if which_dataset == 'sst2':
  ### trata dataset
if which_dataset == 'ag_news':
  raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
  hyp_template = "this text is about {}."



  


Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

## Dataset build

### Train Data

In [181]:
# arq_train = '/content/drive/MyDrive/zeroshot_new_paper/BBC News Train.csv'
# train = pd.read_csv(arq_train)
train = raw_data
len(train)
# train = raw_data[~(raw_data['full_text'].isnull())].copy()
# train = raw_data[~(raw_data['full_text'].isnull())].sample(max_inferences,random_state=42).copy()

127600

In [182]:
labels_list = list(train[class_col].unique())
print(labels_list)

['business', 'science and technology', 'sports', 'world']


# Sentence Extraction

In [183]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

In [184]:
try:
    print(train['sentences'])
except:
    train['sentences'] = train[data_col].progress_apply(lambda txt:tokenizer.tokenize(txt))
    train.head(2)

100%|██████████| 127600/127600 [00:02<00:00, 44911.20it/s]


In [186]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)


127600

In [188]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
# train[['len_sentences']].describe()

100%|██████████| 127600/127600 [00:00<00:00, 1950867.51it/s]


# Zero-Shot

## Data

In [189]:
max_inferences = len(train) #estava em 5000
# max_inferences = 5000

# zeroshot_method = "probability_threshold"
probability_goal = 0.9

zeroshot_method = "top_n_goal"  
top_n_goal = 8

zeroshot_config = {
    'dataset':which_dataset,
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'labels':labels_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422
}
zeroshot_config

{'dataset': 'ag_news',
 'method': 'top_n_goal',
 'prob_goal': 0.8,
 'top_n_goal': 8,
 'max_inferences': 127600,
 'labels': ['business', 'science and technology', 'sports', 'world'],
 'template': 'this text is about {}.',
 'random_state': 422}

In [190]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state'])
len(zeroshot_data)

127600

## Model

In [191]:
classifier = pipeline("zero-shot-classification",
                      model="joeddav/xlm-roberta-large-xnli",device="cpu") # device = 'cuda'

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Inference

In [192]:
%%time
zeroshot_results = zeroberto.runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)
print(len(zeroshot_results))
### 100 predições ---> <240 segundos (4 min)
### 200 predições ---> <480 segundos (8 min)
### 1000 predições ---> <2400 segundos (40 min)
### 5000 predições ---> 12000 segundos (200 min) (3h 20min)

# data: 127600
{'dataset': 'ag_news', 'method': 'top_n_goal', 'prob_goal': 0.8, 'top_n_goal': 8, 'max_inferences': 127600, 'labels': ['business', 'science and technology', 'sports', 'world'], 'template': 'this text is about {}.', 'random_state': 422}
Preds: 50  - Total time: 30.58 seconds - ETA: 1300.63 minutes
Preds: 100  - Total time: 61.83 seconds - ETA: 1314.88 minutes
Preds: 150  - Total time: 90.82 seconds - ETA: 1287.62 minutes
Preds: 200  - Total time: 120.42 seconds - ETA: 1280.43 minutes
Preds: 250  - Total time: 149.57 seconds - ETA: 1272.33 minutes
Preds: 300  - Total time: 182.83 seconds - ETA: 1296.08 minutes
Preds: 350  - Total time: 213.78 seconds - ETA: 1298.95 minutes
Preds: 400  - Total time: 243.35 seconds - ETA: 1293.8 minutes
Preds: 450  - Total time: 273.62 seconds - ETA: 1293.11 minutes
Preds: 500  - Total time: 301.51 seconds - ETA: 1282.44 minutes
Preds: 550  - Total time: 329.5 seconds - ETA: 1274.08 minutes
Preds: 600  - Total time: 359.91 seconds - ETA: 127

KeyboardInterrupt: 

In [169]:
label_probabilities_df = zeroberto.formatZeroShotResults(zeroshot_results)

In [170]:
def getTopLabel(row):
  return row.idxmax()
def getTopProb(row):
  return row.max()

label_results = label_probabilities_df.apply(getTopLabel,axis=1)
prob_results = label_probabilities_df.apply(getTopProb,axis=1)
label_results_df = pd.Series(label_results,name='prediction')

In [171]:
true_labels_df = raw_data[raw_data.index.isin(train.index)][class_col]
final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)

In [172]:
vector_true = ( final_result_df[class_col].str.strip().str.lower() 
               == final_result_df['prediction'].str.strip().str.lower() )

## Metrics

In [173]:
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])

In [174]:
# final_result_df_encoded['predicted?'] = False
# final_result_df_encoded.loc[0:len(zeroshot_results),'predicted?'] = True
# #

In [175]:
evaluation_metrics.get_metrics(final_result_df_encoded['prediction_code'][0:len(zeroshot_results)].to_list(),
                               final_result_df_encoded[class_col+'_code'][0:len(zeroshot_results)].to_list())

{'weighted': [{'accuracy': 0.2280112044817927},
  {'precision': 0.27448665784445037},
  {'recall': 0.2280112044817927},
  {'f1': 0.16360552482264967}],
 'macro': [{'accuracy': 0.2280112044817927},
  {'precision': 0.24425889275722423},
  {'recall': 0.24653616599241013},
  {'f1': 0.17546706781245341}]}

### Probabilities

In [176]:
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

##### Top N results


In [177]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)

    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(zeroshot_results))

{'accuracy in top 1': 0.0}
{'accuracy in top 2': 0.0}
{'accuracy in top 3': 0.083}
{'accuracy in top 4': 0.062}
{'accuracy in top 5': 0.1}
{'accuracy in top 6': 0.083}
{'accuracy in top 7': 0.071}
{'accuracy in top 8': 0.094}
{'accuracy in top 9': 0.083}
{'accuracy in top 10': 0.125}
{'accuracy in top 11': 0.136}
{'accuracy in top 12': 0.146}
{'accuracy in top 13': 0.154}
{'accuracy in top 14': 0.143}
{'accuracy in top 15': 0.15}
{'accuracy in top 16': 0.141}
{'accuracy in top 1785': 0.228}


## Save Results

In [178]:
evaluation_metrics.saveZeroshotResults(zeroshot_config,df_predictions_probs)

predictions_and_probabilities_test_2023_02_13__00_17_37.csv
zeroshot_config_test_2023_02_13__00_17_37.csv


'2023_02_13__00_17_37'

# Load Previous Results

In [30]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [31]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)